In [10]:
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")
charlie = sy.VirtualWorker(hook, id="charlie")
crypto = sy.VirtualWorker(hook, id="crypto_provider")

In [11]:
x = th.tensor([1, 2, 3, 4]).share(alice, bob, charlie,
                                 crypto_provider=crypto)
x

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:50873705501 -> alice:38213365789]
	-> [PointerTensor | me:94727619112 -> bob:986809960]
	-> [PointerTensor | me:65430047594 -> charlie:26254007768]
	*crypto provider: crypto_provider*

In [7]:
import torch as th
import syft as sy
hook = sy.TorchHook(th)

In [2]:
hospital_datacenter = sy.VirtualWorker(hook, id="Curie Institute")

In [3]:
x = th.tensor([1, 2, 3, 4, 5])
x = x.send(hospital_datacenter)
x

(Wrapper)>[PointerTensor | me:15061150250 -> Curie Institute:10756455657]

In [4]:
y = x + x
y

(Wrapper)>[PointerTensor | me:71246720083 -> Curie Institute:9193514890]

In [5]:
y.get()

tensor([ 2,  4,  6,  8, 10])

In [6]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import syft as sy

hook = sy.TorchHook(th)

alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")
charlie = sy.VirtualWorker(hook, id="charlie")
dan = sy.VirtualWorker(hook, id="dan")
james = dan

hook.local_worker.is_client_worker = False
me = hook.local_worker
from syft.generic.pointers.pointer_plan import PointerPlan

import sys
torch = th
syft = sy
#print(sy.hook.local_worker.get_worker('alice')._objects)

In [20]:
def inverse(self, method="NR", nr_iters=10, log_iters=1, all_pos=False):
    #print('self', self.virtual_get())
    if not all_pos:
        sgn = 2 * (self >= 0) - 1
        #print('sgn', sgn.virtual_get())
        abs = sgn * self
        return sgn * abs.inverse(
            method=method, nr_iters=nr_iters, log_iters=log_iters, all_pos=True
        )

    if method == "NR":
        # Initialization to a decent estimate (found by qualitative inspection):
        #                1/x = 3exp(.5 - x) + 0.003
        print('self', self.virtual_get())
        r = (0.5 - self).refresh().exp()
        self_e = torch.tensor([8.3])
        r_exp = (0.5 - self_e).exp()
        result = 3 * r + 0.003
        print(r.virtual_get(), r_exp)
        for _ in range(nr_iters):
            result = result - (result ** 2) * self
        return result
    elif method == "log":
        return (-self.log(iterations=log_iters)).exp()
    elif method == "snn":
        one = self * 0 + 1
        return securenn.division(one, self)
    else:
        raise ValueError("Invalid method %s given for reciprocal function" % method)

In [21]:
sy.AdditiveSharingTensor.inverse=inverse

In [22]:
fix_prec_tolerance = {
    3: 20 / 100,
}

for prec_frac, tolerance in fix_prec_tolerance.items():
    cumsum = torch.zeros(1)
    for i in range(10):
        t = torch.tensor([8.3])
        t_sh = t.fix_precision(precision_fractional=prec_frac).share(alice, bob, crypto_provider=james)
        s = torch.tensor([2.3])
        s_sh = t.fix_precision(precision_fractional=prec_frac).share(alice, bob, crypto_provider=james)
        r_sh = t_sh / s_sh
        r = r_sh.get().float_prec()
        t = t / s
        diff = (r - t).abs()
        norm = (r + t) / 2
        print(r, t)
        cumsum += diff / (tolerance * norm)

    cumsum /= 10
    print(cumsum)
    #assert (cumsum.abs() < 1).all()

self tensor([8300])
tensor([1650484750965399553]) tensor([0.0004])
tensor([-3.2911e+14]) tensor([3.6087])
self tensor([8300])
tensor([1650484750965399553]) tensor([0.0004])
tensor([-3.2911e+14]) tensor([3.6087])
self tensor([8300])
tensor([0]) tensor([0.0004])
tensor([0.]) tensor([3.6087])
self tensor([8300])
tensor([1650484750965399553]) tensor([0.0004])
tensor([-3.2911e+14]) tensor([3.6087])
self tensor([8300])
tensor([0]) tensor([0.0004])
tensor([0.]) tensor([3.6087])
self tensor([8300])
tensor([0]) tensor([0.0004])
tensor([0.]) tensor([3.6087])
self tensor([8300])
tensor([0]) tensor([0.0004])
tensor([0.]) tensor([3.6087])
self tensor([8300])
tensor([1650484750965399553]) tensor([0.0004])
tensor([-3.2911e+14]) tensor([3.6087])
self tensor([8300])
tensor([0]) tensor([0.0004])
tensor([0.]) tensor([3.6087])
self tensor([8300])
tensor([0]) tensor([0.0004])
tensor([0.]) tensor([3.6087])
tensor([2.])


In [149]:
def test(prec, n_iterations):
    deps = torch.zeros(6)
    tolerance = 5 / 100
    for i in range(10):
        th.n_iterations = n_iterations
        #print('---')
        t = torch.tensor([0., 1., 2., 3., 4., 5.])
        t_sh = t.fix_precision(precision_fractional=prec).share(alice, bob, crypto_provider=james)
        r_sh = t_sh.exp()
        r = r_sh.get().float_prec()
        t = t.exp()
        diff = (r - t).abs()
        norm = (r + t) / 2
        depass = diff / (tolerance * norm)
        #print(depass, r, t)
        deps += depass
    print(deps / 10)

In [150]:
t = torch.tensor([5.])
t.exp()

tensor([148.4132])

In [151]:
for prec in [6]:
    for n_it in [6, 7, 8, 9, 10]:
        print(prec, n_it)
        test(prec, n_it)

6 6
tensor([0.0000, 0.0378, 0.1560, 0.3503, 0.6186, 0.9615])
6 7
tensor([0.0000, 0.0397, 0.1552, 0.3486, 0.6185, 0.9648])
6 8
tensor([0.0000, 0.0400, 0.1555, 0.3493, 0.6185, 0.9635])
6 9
tensor([0.0000, 0.0379, 0.1565, 0.3493, 0.6185, 0.9620])
6 10
tensor([0.0000, 0.0390, 0.1555, 0.3486, 0.6185, 0.9650])


In [55]:
test(3, 8)

tensor([0.8545])


In [ ]:
t = torch.tensor([1.0, -2.0, 3.0])
self = t.fix_prec().share(alice, bob, crypto_provider=james)
one = torch.tensor([1.0]).fix_precision()
result = 1 + self.div(2 ** 8)# + 1
result.child.child.virtual_get()

In [5]:
t = torch.tensor([1.0, -2.0, 3.0])
x = t.fix_prec().share(alice, bob, crypto_provider=james)
c = 4

z = (x + c).get().float_prec()
assert (z == (t + c)).all()

z = (c + x).get().float_prec()
assert (z == (c + t)).all()

# with constant float
t = torch.tensor([1.0, -2.0, 3.0])
x = t.fix_prec().share(alice, bob, crypto_provider=james)
c = 4.2

z = (x + c).get().float_prec()
assert ((z - (t + c)) < 10e-3).all()

z = (c + x).get().float_prec()
assert ((z - (c + t)) < 10e-3).all()

In [2]:
t = torch.tensor([1.0, -2.0, 3.0])
x = t.fix_prec()
c = 4

z = (x - c).float_prec()
assert (z == (t - c)).all()

z = (c - x).float_prec()
print(z)
print(c - t)
assert (z == (c - t)).all()

tensor([3., 6., 1.])
tensor([3., 6., 1.])


In [5]:
# with constant float
t = torch.tensor([1.0, -2.0, 3.0])
x = t.fix_prec()
c = 4.2

z = (x + c).float_prec()
print(z - (t + c))
print(t + c)
assert (z == (t + c)).all()

z = (c + x).float_prec()
assert (z == (c + t)).all()

oself, ther tensor([               1000, 4611686018427385904,                3000]) 4.2
scale
tensor([0.0000e+00, 2.3842e-07, 0.0000e+00])
tensor([5.2000, 2.2000, 7.2000])


AssertionError: 

In [2]:
def log(self, iterations=2, exp_iterations=8):

    # Initialization to a decent estimate (found by qualitative inspection):
    # ln(x) = x/40 - 8exp(-2x - .3) + 1.9
    print(self)
    c1 = torch.tensor([0.3])
    c2 = torch.tensor([1.9])
    y = self / 40 - 8 * (-2 * self - c1).exp() + c2
    print('y0', y)

    # 6th order Householder iterations
    for i in range(iterations):
        print(f"y{i}", y)
        e = (-y).exp()
        print("e", e)
        h = 1 - self * e
        print('h', h)
        h2 = h ** 2
        h3 = h2 * h
        h4 = h2 ** 2
        h5 = h4 * h
        u = h * (1 + h.div(2) + h2.div(3) + h3.div(6) + h4.div(5) + h5.div(7))
        print(f"update", u)
        y -= u

    return y

In [3]:
t = torch.tensor([24.])
log(t)

tensor([24.])
y0 tensor([2.5000])
y0 tensor([2.5000])
e tensor([0.0821])
h tensor([-0.9700])
update tensor([-0.7090])
y1 tensor([3.2090])
e tensor([0.0404])
h tensor([0.0305])
update tensor([0.0309])


tensor([3.1781])

In [7]:
t = torch.tensor([24.])
t_sh = t.fix_precision(precision_fractional=3).share(alice, bob, crypto_provider=james)
r_sh = t_sh.log()
r = r_sh.get().float_prec()
print('--')
print(r)
print(t.log())
diff = (r - t.log()).abs()
norm = (r + t) / 2

diff/norm

--
tensor([3.1880])
tensor([3.1781])


tensor([0.0007])

In [35]:
tolerance = 5 / 100
t = torch.tensor([1.4])
t_sh = t.fix_precision().share(alice, bob, crypto_provider=james)
r_sh = t_sh.log()
r = r_sh.get().float_prec()
diff = (r - t.log()).abs()
norm = (r + t) / 2
print(r, t.log())
assert (diff < tolerance * norm).all()

tensor([0.2950]) tensor([0.3365])


In [3]:
t = torch.tensor([1.5]).fix_precision().share(alice, bob, crypto_provider=james)
self = t
c1 = torch.tensor([0.3]).fix_precision()
c2 = torch.tensor([1.9]).fix_precision()
y = self / 40 - 8 * (-2 * self - c1).exp() + c2
y.get().float_prec()

tensor([1.6570])

In [37]:
t = torch.tensor([1.5])
self = t
self / 40 - 8 * (-2 * self - 0.3).exp() + 1.9

tensor([1.6424])

In [36]:
torch.tensor([1.5]).fix_precision().share(alice, bob, crypto_provider=james)

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:34444351479 -> alice:64908857738]
	-> [PointerTensor | me:87436165845 -> bob:43570879381]
	*crypto provider: dan*

In [3]:
t_sh = t_fp.share(alice, bob, crypto_provider=james)

In [4]:
t_sh

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:54663139331 -> alice:40221567173]
	-> [PointerTensor | me:24387318875 -> bob:295315150]
	*crypto provider: dan*

In [5]:
iterations = 8
r_sh = t_sh.div(2 ** iterations) + 1
r_sh.get()

(Wrapper)>FixedPrecisionTensor>tensor([1006])

In [6]:
1500/2**8

5.859375

In [7]:
r_sh = t_sh.exp()

tensor([1500])
tensor([1006])
tensor([1012])
tensor([1024])
tensor([1048])
tensor([1098])
tensor([1206])
tensor([1454])
tensor([2114])
tensor([4469])


In [8]:
r_sh.get().float_prec()

tensor([4.4690])

In [9]:
t.exp()

tensor([4.4817])